# Battery Temperature Prediction using Neural Network Model

This notebook demonstrates the process of predicting battery temperature using a neural network model in TensorFlow. The dataset includes information on voltage, current, state, time, and measured temperature. The data is preprocessed, and a neural network model is trained to predict temperature values. After training, the model is converted to TensorFlow Lite format for inference on mobile or embedded devices.


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# 1.load datasets
data = pd.read_csv('dataset.csv')

In [3]:
#2. Data Preprocessing
data = data[['Voltage_measured (Volts)', 'Current_measured (Amps)', 'state', 'Time (secs)', 'Temperature_measured (C)']]



In [4]:
# Convert 'state' feature to one-hot encoding
data = pd.get_dummies(data, columns=['state'], drop_first=True)
# Convert state to float (0.0 or 1.0)
data['state_discharge'] = data['state_discharge'].astype(float)


In [ ]:
print(data[:4])

In [6]:
# Split data into features (X) and target (y)
X = data.drop('Temperature_measured (C)', axis=1)
y = data['Temperature_measured (C)']


In [7]:
X_scaled = X.values  # بدون نرمال‌سازی

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1) 
])
# 4. Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# 5. Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')


In [ ]:
# 6. Predict temperature and compare with actual values
y_pred = model.predict(X_test)

In [ ]:
# 7. Plot comparison chart
plt.figure(figsize=(10,6))
plt.plot(y_test.values, label='true values',)
plt.plot(y_pred, label="predicted values", linestyle='--')
plt.xlabel('samples')
plt.ylabel('temp')
plt.legend()
plt.title("compare true values and predicted values")
plt.show()

In [ ]:
# Convert model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save TensorFlow Lite model to file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Initialize TFLite interpreter
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Initialize TFLite interpreter
print("TensorFlow Lite Ops in model:")
for op in interpreter._get_ops_details():
    print(op)


In [ ]:
# Load test data for inference
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load test data for inference
data = pd.read_csv("test_data.csv")
data = data[['Voltage_measured (Volts)', 'Current_measured (Amps)', 'state', 'Time (secs)']]
data = pd.get_dummies(data, columns=['state'], drop_first=True)
data['state_discharge'] = data['state_discharge'].astype(float)

X_scaled = data.values  # Original data without scaling


# Inference on test data
for i in range(100):
    input_data = np.array(X_scaled[i], dtype=np.float32).reshape(1, -1)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Invoke the model
    interpreter.invoke()

    # Get predicted temperature
    predicted_temperature = interpreter.get_tensor(output_details[0]['index'])[0][0]
    print(f"Sample {i+1}: Predicted Temperature = {predicted_temperature:.2f}")
    